In [7]:
# Import libraries
import sys
sys.path.append('../')
import numpy as np 
import pandas as pd 
import os
import warnings
import torch
from torch import nn
from torchvision import datasets, transforms, models
from torchvision.models import resnet50
from torch.autograd import Variable
from matplotlib import pyplot as plt
from scipy.spatial import cKDTree
from sklearn.exceptions import ConvergenceWarning
import sklearn.linear_model as slm
import sklearn.feature_selection as skf
import sklearn.preprocessing as skp
import util
import cv2
import pickle

In [2]:
# Augment with CHH data
X0_gt = np.load('/home/ali/RadDBS-QSM/data/npy/old/X0_gt_chh_rois.npy')
df = pd.read_csv('/home/ali/RadDBS-QSM/data/xlxs/updrs_iii_chh.csv')
# Patient IDs
subject_id = np.asarray(df[df.columns[0]])[1:]
n_rois = 6
# Data
s_directory = open('/home/ali/RadDBS-QSM/data/roi/roi_list','r').read().splitlines()
# Load
with open('/home/ali/RadDBS-QSM/data/pickles/segs_chh', "rb") as fp:  
    segs = pickle.load(fp)
    n_cases = len(segs)
with open('/home/ali/RadDBS-QSM/data/pickles/qsms_chh', "rb") as fp:  
    qsms = pickle.load(fp)
with open('/home/ali/RadDBS-QSM/data/phi/chh/Phi_mcl_gt_roi_chh', "rb") as fp:  
    Phi_gt = pickle.load(fp)
L = int(len(X0_gt)/n_cases)
n_features = int(L/n_rois)
# Only extract ROI if it is present in all cases
seg_labels_all = segs[0]
case_number = np.zeros_like(np.asarray(s_directory))
for i in range(n_cases):
    case_number[i] = float(s_directory[i][-2:])
subject_id_corr = subject_id[np.in1d(subject_id,case_number)]
for i in range(n_cases):
    try:
        print('Found ROIs',str(np.unique(segs[i])),'at segmentation directory file',s_directory[i],'for case',str(subject_id_corr[i]))
    except:
        print('Case',subject_id[i],'quarantined')
pre_updrs_iii_off =  np.asarray(df[df.columns[3]][1:][np.in1d(subject_id,subject_id_corr)]).astype(float)                             
pre_updrs_iii_on =  np.asarray(df[df.columns[4]][1:][np.in1d(subject_id,subject_id_corr)]).astype(float) 
post_updrs_iii_off =  np.asarray(df[df.columns[6]][1:][np.in1d(subject_id,subject_id_corr)]).astype(float) 

per_change = (pre_updrs_iii_off-post_updrs_iii_off)/pre_updrs_iii_off
pre_updrs_off = pre_updrs_iii_off
X_all_c = X0_gt.reshape(n_cases,n_rois,n_features)
X_all_c = X_all_c[:,0:4,:]

lct_change = (pre_updrs_iii_off-pre_updrs_iii_on)/pre_updrs_iii_off
pre_imp = lct_change
X_all_c = X_all_c.reshape(X_all_c.shape[0],-1)
X_all_c = np.append(X_all_c,pre_imp.reshape(-1,1),axis=1)
subsc = subject_id_corr

Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000001 for case 1.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000002 for case 2.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000003 for case 3.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000004 for case 4.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000005 for case 5.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000006 for case 6.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000007 for case 7.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000008 for case 8.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000009 for case 9.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000010 for case 10.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segmentation directory file 00000011 for case 11.0
Found ROIs [0. 1. 2. 3. 4. 5. 6.] at segm

In [3]:
pt_model = resnet50(weights='DEFAULT')
for param in pt_model.parameters():
    param.required_grad = False

In [9]:
pt_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
net_add = pt_model
model = nn.Sequential(nn.Linear(64*64,64*64), net_add, nn.Linear(1000,1))
model

Sequential(
  (0): Linear(in_features=4096, out_features=4096, bias=True)
  (1): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU

In [15]:
subs_init = subsc
X_img, subsc0, per_change0, pre_metric0, pre_imp0 = util.slice_pick(subsc=subs_init,
                        per_change=per_change,
                        pre_metric=pre_updrs_off,
                        pre_comp=pre_imp,
                        pshape=(64,64,64),
                        roi_l=0,roi_u=4,
                        mask_crop_output=False,mask_output=False,o_index=False,
                        file_path='/home/ali/RadDBS-QSM/src/jupyter/mlp/torch/X_chh.pt',
                        qsm_path='/home/ali/RadDBS-QSM/data/nii/chh/qsm/',
                        seg_prefix='/home/ali/RadDBS-QSM/data/nii/chh/seg/',
                        save_image=False,
                        L=10,
                        new_prefix=False,
                        img_directory='/home/ali/RadDBS-QSM/src/jupyter/mlp/chh_tensor_slices_0/',
                        visualize=False,
                        reload=True)
Xr = torch.zeros((len(X_img),X_img[0].shape[0],X_img[0].shape[1]))
for j in np.arange(len(X_img)):
    img = X_img[j]
    img[img>250] = 250
    img[img<-250] = -250
    Xr[j,:,:] = img
    #plt.imshow(np.rot90(img.cpu()),cmap="gray")
    #plt.show()
    np.save('./chh_tensor_slices_0/case_'+str(subsc[j])+'_0.npy',img.cpu())

1
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_01.nii.gz
Maximum volume found at slice 33 for case 1
2
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_02.nii.gz
Maximum volume found at slice 35 for case 2
3
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_03.nii.gz
Maximum volume found at slice 33 for case 3
4
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_04.nii.gz
Maximum volume found at slice 34 for case 4
5
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_05.nii.gz
Maximum volume found at slice 34 for case 5
6
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_06.nii.gz
Maximum volume found at slice 33 for case 6
7
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_07.nii.gz
Maximum volume found at slice 35 for case 7
8
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_08.nii.gz
Maximum volume found at slice 33 for case 8
9
/home/ali/RadDBS-QSM/data/nii/chh/qsm/QSM_e10_imaginary_09.nii.gz
Maximum volume found at slice 34 for

KeyboardInterrupt: 

In [ ]:
util.get_layer_output(X_img[j],pt_model,{"layer1.0.conv1": "layer4"})['layer4'].detach().numpy().shape

In [ ]:
Z = []
zc1 = []
x = []
for j in np.arange(len(X_img)):
    x_img = X_img[j]
    x_img[x_img>250] = 250
    x_img[x_img<-250] = -250
    x.append(x_img.cpu().numpy())
    Z.append(util.get_latent_rep(X_img[j],pt_model).detach().numpy())
    zc1.append(util.get_layer_output(X_img[j],pt_model,{"layer4.2.conv3": "layer4"})['layer4'].detach().numpy())
    plt.imshow(Z[j].reshape((10,100)))

In [ ]:
X = np.asarray(x)
Zc1 = np.asarray(zc1)

In [ ]:
# plt.figure(figsize=(9, 5), dpi=100)

# plt.imshow(np.rot90(np.hstack((X[0:9,:].reshape((-1,64)),X[9:18,:].reshape((-1,64)),
#                       X[18:27,:].reshape((-1,64)),X[27:36,:].reshape((-1,64))))),cmap='gray')
#                     #  X[36:45,:].reshape((-1,64))))),cmap='gray')
                     
# plt.style.use('dark_background')
# plt.axis('off')


In [ ]:
Z = np.asarray(Z)

In [ ]:
r = np.zeros(len(per_change))
err_var = np.zeros(len(per_change))
rerror = np.zeros(len(per_change))
kappa = []
results_ls = np.zeros(len(per_change))
scaler = skp.StandardScaler()
Z = np.asarray(Zc1.reshape(Zc1.shape[0],-1))
Z = scaler.fit_transform(np.squeeze(Z))
for j in np.arange(len(subsc)):
    test_id = subsc[j]
    test_index = subsc == test_id
    train_index = subsc != test_id
    X_train = Z[train_index]
    X_test = Z[test_index]
    y_train = per_change[train_index].ravel()
    y_test = per_change[test_index]

    X0_ss0 = X_train.reshape(X_train.shape[0],-1)
    X_test_ss0 = X_test.reshape(X_test.shape[0],-1)
    # Cross validation
    cvn = 5
    # X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
    #                                             X_train,train_index,X_test,test_index,pre_metric,ledd,False,False,False)
    lasso = slm.LassoCV(max_iter=1000,cv=cvn,n_jobs=-1,verbose=1)

    with warnings.catch_warnings() and np.errstate(divide='ignore', invalid='ignore'):
      # Feature selection
      warnings.filterwarnings("ignore", category=ConvergenceWarning)
      #sel = skf.SelectKBest(skf.r_regression,k=500)
      X0_ss = X0_ss0#sel.fit_transform(X0_ss0,y_train)
      X_test_ss = X_test_ss0#sel.transform(X_test_ss0)
      est_ls = lasso.fit(X0_ss,y_train)
    results_ls[j] = est_ls.predict(X_test_ss)
      
    print('Lasso predicts',str(np.round(results_ls[j],2)),
              'for case with',str(np.round(per_change[j],2)))
              #'with regularization',str(est_ls.alpha_))

In [ ]:
util.eval_prediction(np.vstack((np.squeeze(pre_imp),
                               results_ls.T,
                               )),
                               np.squeeze(per_change),
                               ['LCT',
                                'Lasso',
                                ],(20,10))
plt.ylim([0,1.5])
plt.xlim([0,1.5])
plt.style.use('default')